# 利用拓扑数据分析中持续同调来检测数据中的异常值。
- 该方法基于数据的拓扑信息来检测异常值，计算代表正常数据集的持续图与添加了异常值后数据集的持续图之间的距离值
- 当该值大于某一个阈值时，认为添加的数据点是异常值，小于该阈值时，认为添加的数据点是正常值
- 这里需要解决两个首要问题：
1. 如何选择代表整个数据集形状的数据集，如果选择整个正常数据集，会导致数据量较大，计算量大，使得空间复杂度和时间复杂度比较高；
2. 正常数据集较多会导致冗余，有可能降低检测或预测的准确度，类似于机器学习模型的过拟合。
- 机器学习方法进行异常检测是通过学习得到模型，拓扑数据分析方法进行异常检测是通过学习得到代表整个正常数据集的拓扑模型（即，数据的形状）

## 导入所需的库和包

In [2]:
import gudhi as gh
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from tqdm import tqdm
import time
import sys
sys.path.append('../.')
from TimeStamps import display_time
import copy

## 定义拓扑类，用于计算持续同调和 bottleneck 距离

In [3]:
class Topology(object):
    
    def __init__(self, data, max_edge_length=42, max_dimension=1, homology_coeff_field=2, min_persistence=0):
        self.data = data
        self.max_edge_length = max_edge_length
        self.max_dimension = max_dimension
        self.homology_coeff_field = homology_coeff_field
        self.min_persistence = min_persistence
        
    def ph(self, plot=True):
        
#         print("RipsComplex creation from points")
        data = preprocessing.scale(self.data)
        rips = gh.RipsComplex(points=data, max_edge_length=self.max_edge_length)
        simplex_tree = rips.create_simplex_tree(max_dimension=self.max_dimension)
        diag = simplex_tree.persistence(homology_coeff_field=self.homology_coeff_field, 
                                        min_persistence=self.min_persistence)
#         print("diag=", diag)
        
        if plot == True:
            pplot = gh.plot_persistence_diagram(diag)
            pplot.show()
        return diag
    
    def bottleneck(self, point, plot=True):
        diag = self.ph(plot)
        diag0 = []
        for i, d in enumerate(diag):
            diag0.append(list(diag[i][1]))
        
        temp_data = self.data
        self.data = np.vstack((self.data, point))
        diag_point = self.ph(plot)
        diag_point0 = []
        for i, d in enumerate(diag_point):
            diag_point0.append(list(diag_point[i][1]))
            
        self.data = temp_data

        return gh.bottleneck_distance(diag0, diag_point0)
    
    @display_time
    def grow(self, input_path, outlier_number, output_path, threshold=1.0, base_index=10, tree_height=100):
        '''
        param input_path: pandas数据集的路径
        param outlier_number: 数据集中包含的异常点数
        param output_number: 找到的形状数据集B的输出路径
        param threshold: 判断是否保留随机选中的数据点的阈值，大于该阈值丢弃该点，小于该阈值保留该点
        param base_index: 期初形状数据集B的点的个数
        param tree_height: 生长形状数据集B的最大个数
        '''
        satellite = pd.read_csv(input_path, header=None)
        satellite_list = list(np.arange(outlier_number, len(satellite.index)))
        if base_index > len(satellite_list):
            print('out of bound about base_index')
            
        else:
            base_list = list(np.random.choice(satellite_list, base_index, replace=False))
            self.data = np.array(satellite.iloc[base_list, :-1])
            for i in tqdm(np.arange(outlier_number, len(satellite.index))): 
                # 从第85个数据开始验证并生长形状
                point = np.array(satellite.iloc[i, :-1]) # 索引出每一个待验证的数据点

                if i in base_list:
                    continue
                else:
                    if len(self.data) < tree_height:
                        if self.bottleneck(point, plot=False) <= threshold:
                            # 以阈值1为验证条件，小于1的添加到B数据中，大于1的舍弃
                            base_list.append(i) # 将生长的数据索引取出
                            self.data = np.vstack((self.data, point)) # 将通过验证的数据进行生长
                    else:
                        break

            B = satellite.iloc[base_list, :] # 生成的形状数据B
            B.to_csv(output_path) # 将长成的形状数据保存1起来
#             print('base_list: ', base_list)
            print('self data shape: ', self.data.shape[0]) # 打印计算中使用的生长数据的形状
            print('B data shape: ', B.shape) # 打印出真实生长数据的形状
            
    @display_time
    def crop(self, input_path, outlier_number, output_path, threshold=1.0, base_lower=10, cycle=100):
        '''
        param input_path: pandas数据集的路径
        param outlier_number: 数据集中包含的异常点数
        param output_number: 找到的形状数据集B的输出路径
        param threshold: 判断是否保留随机选中的数据点的阈值，大于该阈值丢弃该点，小于该阈值保留该点
        param base_lower: 形状数据集B的最小点数
        param cycle: 随机筛选数据集的次数
        '''
        satellite = pd.read_csv(input_path, header=None)
        self.data = np.array(satellite.iloc[outlier_number:, :-1])
        base_list = list(np.arange(outlier_number, len(satellite.index)))
        if base_lower > len(self.data): # 如果基数集的个数设置的过大，超过A的点数，则返回
            print('base lower is out of bound')
        else: # 如果不超过A的点数，则继续
            for i in tqdm(np.arange(cycle)): # 循环随机从现有的数据集索引中筛选数据点
                choice_index = np.random.choice(base_list, 1) # 随机筛选一个点，以相同概率
                point = np.array(satellite.iloc[choice_index, :-1]) # 找到该索引对应的数据点
                temp_list = copy.deepcopy(base_list) # 保留现在的索引列表，以防上面筛选的点没有被丢弃
                base_list.remove(choice_index) # 暂时丢弃筛选的点，看看是否导致形状改变很大
                self.data = np.array(satellite.iloc[base_list, :-1]) # 给出剔除筛选数据点后的数据集
                if len(self.data) < base_lower: # 如果提出后基数集B个数小于设定的界限，则停止循环，直接返回
                    break
                else: # 没有低于最小界限，则继续
                    if self.bottleneck(point, plot=False) <= threshold:
                        # 如果选中的点能够有效代表整个数据集的形状，则保留
                        base_list = copy.deepcopy(temp_list)
                        self.data = np.array(satellite.iloc[base_list, :-1])
                    else: # 如果不能，则剔除该点
#                         base_list = copy.deepcopy(base_list)
#                         self.data = np.array(satellite.iloc[base_list, :-1])
                        continue

            B = satellite.iloc[base_list, :] # 生成的形状数据B
            B.to_csv(output_path) # 将长成的形状数据保存1起来
#             print('base_list: ', base_list)
            print('self data shape: ', self.data.shape[0]) # 打印计算中使用的生长数据的形状
            print('B data shape: ', B.shape) # 打印出真实生长数据的形状
            
    @display_time
    def reduce(self, input_path, outlier_number, output_path, threshold=1.0, base_lower=10):
        '''
        param input_path: pandas数据集的路径
        param outlier_number: 数据集中包含的异常点数
        param output_number: 找到的形状数据集B的输出路径
        param threshold: 判断是否保留随机选中的数据点的阈值，大于该阈值丢弃该点，小于该阈值保留该点
        param base_lower: 形状数据集B的最小点数
        '''
        satellite = pd.read_csv(input_path, header=None)
        self.data = np.array(satellite.iloc[outlier_number:, :-1])
        base_list = list(np.arange(outlier_number, len(satellite.index)))
        if base_lower > len(self.data): # 如果基数集的个数设置的过大，超过A的点数，则返回
            print('base lower is out of bound')
        else: # 如果不超过A的点数，则继续
            for i in tqdm(np.arange(outlier_number, len(satellite.index))): # 循环随机从现有的数据集索引中筛选数据点
                choice_index = i # 从前往后，一个一个排查
                point = np.array(satellite.iloc[choice_index, :-1]) # 找到该索引对应的数据点
                temp_list = copy.deepcopy(base_list) # 保留现在的索引列表，以防上面筛选的点没有被丢弃
                base_list.remove(choice_index) # 暂时丢弃筛选的点，看看是否导致形状改变很大
                self.data = np.array(satellite.iloc[base_list, :-1]) # 给出剔除筛选数据点后的数据集
                if len(self.data) < base_lower: # 如果提出后基数集B个数小于设定的界限，则停止循环，直接返回
                    break
                else: # 没有低于最小界限，则继续
                    if self.bottleneck(point, plot=False) <= threshold:
                        # 如果选中的点能够有效代表整个数据集的形状，则保留
                        base_list = copy.deepcopy(temp_list)
#                         self.data = np.array(satellite.iloc[base_list, :-1])
                    else: # 如果不能，则剔除该点
#                         base_list = base_list
#                         self.data = np.array(satellite.iloc[base_list, :-1])
                        continue

            B = satellite.iloc[base_list, :] # 生成的形状数据B
            B.to_csv(output_path) # 将长成的形状数据保存1起来
#             print('base_list: ', base_list)
            print('self data shape: ', self.data.shape[0]) # 打印计算中使用的生长数据的形状
            print('B data shape: ', B.shape) # 打印出真实生长数据的形状

## 解决问题1：如何找到一个只包含正常点的数据集B表示整个数据集A的形状信息？
- 首先，选择10个点作为种子，并设置一个阈值T（与数据集相关），比如T=0.1；
- 其次，迭代计算其他正常点加入后是否超过阈值，如果超过阈值则不将该点加入B，如果小于阈值则加入B；
- 最后，遍历完所有的正常点数据，如果数据集A的势太大，可以提前结束遍历，结束的条件可以是
1. #（B）=100
2. 遍历完A
- 找到B后，就是整个数据集A的基本数据集，它代表了A的数据拓扑信息。

## 进行验证

## 解决问题2：如何找到一个合适的阈值？
1. 根据筛选B数据集的阈值，作为参考，可以选择比该阈值大的值作为异常检测的阈值；
2. 也可以通过多次筛选B数据集来查看一系列的阈值，找到最大的阈值Tmax能够使得B=A，使用Tmax作为异常检测的阈值；

#### 找不同阈值得到的形状数据集，用来下一步检测异常数据给出一些参考阈值，这里使用reduce的从前往后的顺序裁剪方法，而不是随机裁剪，并且保留75个异常数据和前75个正常数据，用以后面的异常检测验证

In [ ]:
# threshold_list = [0.25, 0.5, 0.75, 0.85, 0.95, 1, 1.05, 1.10, 1.15, 1.20]
threshold_list = list(np.linspace(0.35, 0.5, 6))

for threshold in threshold_list:
    input_path = './data/satellite-unsupervised-ad.csv'
    satellite = pd.read_csv(input_path)
    outlier_number = 150
    base_lower = 100
    data = np.array(satellite.iloc[outlier_number:outlier_number + 10, :-1])
    output_path = './data-base-shape-t/reduce-base-basLow'+ str(base_lower) + '-thrVal' + str(threshold) + '.csv'
    top = Topology(data, max_dimension=1)
    top.reduce(input_path=input_path, outlier_number=outlier_number, output_path=output_path, threshold=threshold, 
             base_lower=base_lower)

#### 找不同阈值得到的形状数据集，用来下一步检测异常数据给出一些参考阈值，这里使用 grow 的随机生长的方法，并且保留75个异常数据和前75个正常数据，用它们作为的异常检测的验证数据集

In [ ]:
# threshold_list = [0.25, 0.5, 0.75, 0.85, 0.95, 1, 1.05, 1.10, 1.15, 1.20]
threshold_list = list(np.linspace(0.25, 2, 15))
for threshold in threshold_list:
    input_path = './data/satellite-unsupervised-ad.csv'
    satellite = pd.read_csv(input_path)
    outlier_number = 150
    base_index = 10
    tree_height = 500
    data = np.array(satellite.iloc[outlier_number:outlier_number + base_index, :-1])
    output_path = './data-base-shape-t/grow-base-treHei'+ str(tree_height) + '-thrVal' + str(threshold) + '.csv'
    top = Topology(data, max_dimension=1)
    top.grow(input_path=input_path, base_index=base_index, outlier_number=outlier_number, tree_height=tree_height,
                output_path=output_path, threshold=threshold)
    print('threshold: ', threshold)

# test: demo - plot persistent diagram

In [7]:
input_path = './data/satellite-unsupervised-ad.csv'

satellite = pd.read_csv(input_path)
data_normal = np.array(satellite.iloc[150:200, :-1])
top = Topology(data_normal, max_dimension=1)
# top.ph()

# S_1 = data_normal
# S_2 = data_normal + a
# ph_diagram_S1 = ph.persistence_diagram(S_1)
# ph_diagram_S2 = ph.persistence_diagram(S_2)
# d = top.bottleneck(ph_diagram_S1, ph_diagram_S2)
# if a is normal:
#     d little one
# ifelse a is novel:
#     d bigger

satellite.head()


,46.0,40.0,119.0,139.0,42.0,30.0,135.0,157.0,42.0.1,30.0.1,...,113.0,50.0.2,46.0.1,111.0,116.0,44.0.1,31.0,131.0,142.0,o
0,47.0,37.0,119.0,133.0,44.0,34.0,124.0,143.0,44.0,34.0,...,85.0,50.0,39.0,118.0,132.0,43.0,29.0,133.0,143.0,o
1,80.0,95.0,100.0,74.0,64.0,64.0,104.0,96.0,46.0,36.0,...,81.0,82.0,91.0,92.0,78.0,78.0,83.0,96.0,74.0,o
2,56.0,51.0,72.0,60.0,59.0,54.0,72.0,60.0,59.0,51.0,...,50.0,57.0,55.0,74.0,61.0,57.0,55.0,78.0,65.0,o
3,44.0,34.0,129.0,140.0,44.0,34.0,124.0,136.0,44.0,34.0,...,139.0,43.0,31.0,128.0,135.0,43.0,29.0,128.0,132.0,o
4,46.0,48.0,108.0,107.0,53.0,75.0,104.0,92.0,64.0,95.0,...,133.0,46.0,32.0,112.0,133.0,46.0,32.0,112.0,133.0,o
